<a href="https://colab.research.google.com/github/FAMAS73/machine-learning/blob/main/kaggle_intents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter

from sklearn.preprocessing import LabelEncoder

# Load intents from JSON file
with open('intents.json') as file:
    data = json.load(file)

# Extract intent tags and patterns
intents = data['intents']
patterns = []
tags = []
responses = []
for intent in intents:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(intent['tag'])
    responses.extend(intent['responses'])

# Tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(patterns)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

# Pad sequences
max_sequence_length = max([len(pattern.split()) for pattern in patterns])
sequences = tokenizer.texts_to_sequences(patterns)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')


# Encode the tags using LabelEncoder
label_encoder = LabelEncoder()
encoded_tags = label_encoder.fit_transform(tags)
# Convert encoded tags to one-hot encoded labels
tag_labels = np.zeros((len(tags), len(set(encoded_tags))))
for i, tag in enumerate(encoded_tags):
    tag_labels[i, tag] = 1

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(padded_sequences, tag_labels, test_size=0.2, random_state=42)

# Display the shapes of the training and validation sets
print("Training data shape:", X_train.shape)
print("Training labels shape:", y_train.shape)
print("Validation data shape:", X_val.shape)
print("Validation labels shape:", y_val.shape)




Training data shape: (185, 18)
Training labels shape: (185, 80)
Validation data shape: (47, 18)
Validation labels shape: (47, 80)


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

# Define RNN model
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_sequence_length))
model.add(SimpleRNN(128))
model.add(Dense(len(set(tags)), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Display the model summary
print(model.summary())


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 18, 128)           38912     
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense_14 (Dense)            (None, 80)                10320     
                                                                 
Total params: 82128 (320.81 KB)
Trainable params: 82128 (320.81 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [21]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/20
6/6 [==============================] - 2s 83ms/step - loss: 4.3777 - accuracy: 0.0216 - val_loss: 4.3652 - val_accuracy: 0.0213
Epoch 2/20
6/6 [==============================] - 0s 20ms/step - loss: 4.1373 - accuracy: 0.1027 - val_loss: 4.2816 - val_accuracy: 0.0426
Epoch 3/20
6/6 [==============================] - 0s 18ms/step - loss: 3.8029 - accuracy: 0.2595 - val_loss: 4.2126 - val_accuracy: 0.0426
Epoch 4/20
6/6 [==============================] - 0s 21ms/step - loss: 3.4589 - accuracy: 0.2757 - val_loss: 3.9990 - val_accuracy: 0.0851
Epoch 5/20
6/6 [==============================] - 0s 20ms/step - loss: 3.0543 - accuracy: 0.4541 - val_loss: 3.9243 - val_accuracy: 0.1064
Epoch 6/20
6/6 [==============================] - 0s 18ms/step - loss: 2.5816 - accuracy: 0.6270 - val_loss: 3.8223 - val_accuracy: 0.1489
Epoch 7/20
6/6 [==============================] - 0s 21ms/step - loss: 2.1619 - accuracy: 0.7784 - val_loss: 3.7781 - val_accuracy: 0.1277
Epoch 8/20
6/6 [===========

In [22]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print("Validation loss:", loss)
print("Validation accuracy:", accuracy)


2/2 [==============================] - 0s 8ms/step - loss: 3.8169 - accuracy: 0.1489
Validation loss: 3.8169100284576416
Validation accuracy: 0.1489361673593521
